## Repaso de segmentado

El segmentado tenia algunos puntos basicos que tenian que cumplirse si o si. La primer cosa que todo sistema de segmentado tiene que tener es que permite la relocacion dinamica de los programas. Para que realmente sea funcional en el sentido de proteger tambien, tendriamos que agregar el limite y la proteccion ante cambios, no solamente que haya una base y un limite sino que las tareas no puedan cambiar la base y el limite. Pero lo mas importante es base, limite y proteccion para que sea un sistema funcional. Despues hay mejora que permiten optimizar el rendimiento del sistema, como poner un segmento presente o ausente y tambien el bit de access que le permite al SO llevar una especie de estadistica de que cosas se estan usando y que no, cosa de que cuando le falte memoria tenga que elegir no al azar que bajar de una forma pensando en funcion de las necesidades del sistema. Pero en definitiva, con base, limite y proteccion ante cambios podriamos tener un sistema de segmentado funcional. Lo que habiamos visto que era muy costoso, tecnicmanete se puede hacer, pero en la practica no es muy realizable es agrandar un segmeto, mover todo a un lugar mas grande, cambiar la base y cambiar el limite y devolverle el control a la tarea. En la practica mover grandes bloques de memoria y copiarlos para otro lado es algo que pierde mucha performance. La idea es que cuando uno hace un task switch la idea es hacerlo cambiando unos pocos registros, entonces si bien es algo tecicnamente posible, no es algo muy realizable cuando hablamos de segmenentado.

## Manejo de memoria

Para poder decir en que lugar va a cargar cada tarea, donde va a poner la base y el limite de cada segmento, a parte de leer el ejecutable, y que dentro de la imagen del ejecutable estuviera detallado cuantos segmentos habia, el SO tiene que tener algun registro de que es lo que tiene usado. Ahi hay escencialmente dos formas de hacerlo. La primera es utilizar bitmaps, como en el extended del filesystem. Ventaja de los bitmaps es que ocupa muy poco espacio, por cada bloque estamos usando un solo bit. La desventaja es que es lento para buscar. Otra alternativa es usar listas. La ventaja es que hay multiples algoritmos de busqueda que permiten buena performance para buscar. Sin embargo dependiendo de que lista utilices puede ser lento para liberar memoria. Cuando uno habla de hacer listas, aparecen dos opciones, una lista de lugar libre o una lista de lo ocupado. En principio puedo usar varios algoritmos, el tema es como voy a alocar el lugar. Primero la first fit: voy a barrer esa lista con todos los elementos que tenga, cada elemento va a representar un blouqe libre, hasta encontrar el primero que tenga tamanio mayor o igual a lo que busco, osea que lo puedo guardar ahi, esta es una alternativa, se le dara mucho uso a las posiciones de memoria baja y poca a las altas. El next fit es una variante del first, en vez de tener una lista encadenada en donde arranco en la primer posicion de memoria y voy hasta la ultima, mantengo una lista circular. Tambien se barre la lista buscando el primer bloque mayor igual al necesario. Luego el best fit, escencialmente consiste en no encontrar el primer lugar que sea mayor igual al que busco, sino que barrer todos y decir cual es el mas chico en que me entra, que tendria menos desperdicio. En ppio parece una buena alternativa pero tiene resultados bastantes negativos. Entonces me encuentro con una cantiadd de memoria que sumada es importante pero la tengo toda repartida en pedacitos. En worst fit se barre la lista completa buscando al mayor bloque mayor igual al necesario. En el quick fit, se manejan mas de una lista. Ninguna es la mejor, alguna tiene mejor performance en alguna situacion y otros mejor en otra. Dependera de los tamanios de los programas, la frecuencia con las que los programas entran y salen, el grado de ocupacion de la memoria. Y los distintos SO optan por un sistema que consideran que es el mejor.

## Memoria virtual

La realidad es que si nosotros sumamos toda la memoria que ocupan los segmentos podriamos tener mas lugar asociado a los procesos que la memoria fisica. Ese proceso de ir bajando a disco las cosas que no uso es lo que se conoce como memory swapping, es un proceso que comienza cuando el SO se empieza a quedar sin memoria, mientras que tenga memoria, se la da los procesos y no conviene bajarla a disco. El SO no espera a que haya 0% de memoria libre para bajar a disco, va mirando y hace lugar con anticipacion. El problema del sistema de segmentado es que si un segmento comienza a crecer y crecer y la tarea le pidiera mas memoria que el total del sistema, nunca se la podria dar. No podria haber nunca un segmento mayor que la totalidad de memoria. Por eso aparece el otro sistema de manejo de memoria, que se llama manejo de memoria virtual, es algo independiente en principio del segmentado mas alla de que se pueda usar en conjunto. Consiste en lo siguiente: todos los procesos ven un enorme bloque de ram que empeiza en la cero hasta la cantidad maxima de memoria que se podria alocar, por ej un procesador hoy en dia tienen 56 bits de adress para la memoria, 2^56 daria varios teras, entonces alguien estaria viendo un bloque de varios teras. En ppio para el programador esta buenisimo porque dice que tiene todo lo que necesita. Este enorme bloque esta dividio en paginas, estas paginas con bloques de tamanio constante, en caso de intel son 4k. Entonces ese enorme bloque de RAM es una sucesion de bloquecitos de 4k, uno debajo del otro. Cada uno de esos bloquecitos de 4k tienen un estado, puede estar no alocado, osea el proceso nunca la solicito y el proceso no tiene asociado una pagina. Por otro lado, puede estar alocado y presente, osea que esa pagina dentro del espacio virtual del proceso realmente se corresponde con una pagina fisica que esta en algun lado. Y puede estar alocada y ausente, es decir estaba alocada, en algun momento correspondio a una pagina fisica, pero se la bajo a disco entonces ahora no esta. Este nuevo sistema se llama paginado porque esta basado en estos bloques de memoria de tamanio constatne que se llaman paginas. Ahora apaece un problema nuevo, esto al igual que segmentado se maneja por hardware. En segmetado para no tener que ir a memoria a buscar la tabla de segemtnos cada vez que accediamos a una posicion de memoria, lo que se hacia era al momento de cargar un registro de segmentacion ahi se iba a la tabla y se copia el registro del procesador la bases el limite, etc. Es un registro oculto uno solo ve el numero de segmento. Una vez que cargue el registro de segmentacion ya tengo todo en el procesador. Ahora, por otro lado, estamos hablando de miles y miles de paginas, para cada pagina tendriamos que tener alguna entrada en algun lado. El tema es donde lo metemos, esa tabla es tan grande que no podemos meterla en registro. Como podemos mantener esto en memoria? La respuesta es un cache especial que tiene el procesador que se llama TLB que contiene algunas de las entradas de la pagina. Este sistema de paginado solo es viable por como funcionan los programas hoy, si el dia de maniana hubiera un cambio radical en la forma de programar seria posible que esto no funcione mas. Porque funciona esto? porque la realidad indica que los programas utilizan una poca cantidad de paginas a la vez y las usa muchas veces. Por esa forma en que trabajan los programas es que esto es viable. Un ejemplo: el primero que utilizo este sistema fue MULTICS pero hoy en dia la mayoria de los procesadores lo utilizan. El TLB de nivel 1 tiene para codigo 128 entradas, asume que un programa no utiliza mas de 128 paginas de 4k cada una en el codigo que esta ejecutando. Para los segmentos de datos tiene 64 entradas. Y tambien tiene un nivel 2, mas lento que nivel 1 pero mas rapido que RAM que tiene 512 entradas, asume que cuando el proceso esta ejecutando, entre codigo y dato no supera los 2 megas, 512 entradas de 4k cada una. Claramente es un tema empiricio, los procesadores van variando en el tiempo, entre ellos se pelea el 1 o 2% de velocidad, entonces esto es una foto en el tiempo. Por supuesto si algo no esta en el cache hay que ir a RAM a buscarla, no queda otra hay que ir a RAM, en cuanto voy a RAM voy a tratar de cachearlo para tenerlo para despues. En algun momento se nos decia que cambiar de thread es menos costoso que cambair de proceso, como los threads comparten la memoria, obviamente usan el mismo cache del mismo TLB, si cambio de thread es probable que el contenido del TLB me sirva, cuando cambio de proceso normalmente el proceso de cambiar de un proceso a otro (task switch) se flashea el TLB (se lo borra por completo) para que empiece a aprender las paginas del nuevo proceso. Que hacer cuando una pagina no esta en el cache depende del procesador. La familia del 8086 se le carga cierta info para que el propio procesdor pueda entrar a la tabla de paginas para bajarse lo que le falta. Hubo otros procesadores que requerian del SO, cuando una pagina no estaba el SO era el que tenia que tener la tabla, agarrarlo, cargarlo en un registro, etc.. esa alternativa es mas lenta porque es por software, en el caso de x86 se hace por hardware. Al igual que los segmentos el sistema de paginado tiene un bit de acces por ende el SO puede saber que pagina se esta usando y cual no, y llegado el caso llevar esa estadistica para cuando necesita memoria y no hay poder bajar ciertas paginas. Tambien tiene que tener un bit de present dado que deseamos que el SO pueda bajar paginas a disco y transferirle el control a la tarea y que nos avise el hardware si la van a usar y si no la van a usar que siga ejecutando. Hay un bit mas que en el x86 no lo tiene en el segmentado pero si en el paginado que es el que la pagina fue modificada, toda operacion de escritura sobre la pagina prende ese bit. En el caso de los segmentos que son read only no tiene mucho sentido y el SO ya puede optimizarlos de una. La interrupcion que genera el sistema de paginado cuando el programa quiere acceder a una pagina que no existe, sea que este en disco o sea que no fue alocada, se conoce como page fault. El SO tiene que ver cual de los dos casos es, si es por no ser alocada o porque no esta presente. Paginado en los procesadores x86, lamentablemnte en el x86 no se puede apagar el segmentado, normalmente el paginado es mas eficiente porque tiene mas granularidad y encima como son todas de tamanio fijo no hay que ir buscando donde entre. Normalmente cuando usamos en un programa una direccion, esa direccion es lo que se conoce como direccion virtual, eso no tiene nada que ver con la direccion fisica. En los procesadores x86, primero, la direccion pasa por el sistema de segmentado. Pero primero se convierte esa direccion logica o virtual en lo que se llama direccion lineal, como se hace? se pasa por el sistema de segmentado, es decir se le suma la base y se fija que no supere el limite. Si el paginado esta apagado, la direccion lineal es la direccion fisica. Si el paginado esta encedido, ahora hay una segunda conversion por el sistema de paginado, en el cual la direccion lineal se va a convertir en la direccion fisica pasando por todas las tablas de paginado. Entonces la direccion virtual o logica primero la transofrma el segmentado, genera la direccion lineal, y si esta habilitado esa direccion lineal es transformada por el paginado. Como se convierte la direccion lineal en direccion fisica? habia dos niveles de tablas, el page directory y el page table. Para hacer la conversion de lineal a fisica se parten los 32 bits en 10 + 10 + 12, cada entrada tanto en el page directory como en el page table tienen la estructrua de la imagen de abajo. Entonces, agarro la direccion y la divido en 10, 10, 12. Con los primeros 10, entro en la tabla, como sabe el procesador donde esta esa tabla? hay un registro llamado cr3 que si el paginado esta activado tiene que ser inicializado en la direccion de memoria fisica que contiene el page direcroty. Con estos 10 bits elije una de las 1024 entradas, ahora esa entrada no me da la direccion fisica, me da la direccion de otra entrada en otra tabla. Con los siguientes 10 bits entro en una de las entrada de esa tabla. 